# Import Dependencies

In [2]:
%pip install numpy<2 
%pip install --upgrade tensorflow
%pip install --upgrade ml_dtypes

Note: you may need to restart the kernel to use updated packages.


The system cannot find the file specified.



  Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl (15.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.1
    Uninstalling numpy-2.1.1:
      Successfully uninstalled numpy-2.1.1


  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
onnxconverter-common 1.14.0 requires protobuf==3.20.2, but you have protobuf 4.25.3 which is incompatible.
tf-agents 0.19.0 requires typing-extensions==4.5.0, but you have typing-extensions 4.12.2 which is incompatible.


  Attempting uninstall: ml_dtypes
    Found existing installation: ml-dtypes 0.3.2
    Uninstalling ml-dtypes-0.3.2:
      Successfully uninstalled ml-dtypes-0.3.2
Note: you may need to restart the kernel to use updated packages.


  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.17.0 requires ml-dtypes<0.5.0,>=0.3.1, but you have ml-dtypes 0.5.0 which is incompatible.


In [1]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import os
import librosa
import numpy as np
import moviepy.editor as mp
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
import random
import wave
import logging
import werkzeug
import base64
from io import BytesIO

# Model Configuration

In [2]:
logging.basicConfig(level=logging.DEBUG)

custom_object = {
    'LeakyReLU': tf.keras.layers.LeakyReLU,
    'Functional': tf.keras.Model
}

# Data Preprocessing

In [3]:
def preprocess_image(img):
    img = cv2.resize(img, (256, 256))  
    img = img.astype(np.float32) / 255.0
    img = np.expand_dims(img, axis=0)
    return img

In [4]:
def preprocess_spectrogram(spectrogram):
    image = tf.image.resize(spectrogram, (124, 129))
    image = tf.image.grayscale_to_rgb(image)
    image = tf.expand_dims(image, axis=0)  
    return image

In [5]:
def load_wav_16k_mono(audio_file):
    audio_tensor = tf.io.read_file(audio_file)
    audio_tensor = tf.audio.decode_wav(audio_tensor, desired_channels=1).audio
    audio_tensor = tf.squeeze(audio_tensor, axis=-1)
    audio_tensor = tf.cast(audio_tensor, tf.float32)
    return audio_tensor

In [6]:
def extract_frames(video_path, num_frames=10):
    cap = cv2.VideoCapture(video_path)
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    selected_frame_indices = sorted(random.sample(range(total_frames), num_frames))
    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if frame_count in selected_frame_indices:
            frames.append(frame)
        frame_count += 1
    cap.release()
    return frames

In [7]:
def extract_audio(video_path):
    if not os.path.isfile(video_path):
        raise FileNotFoundError(f"The file {video_path} could not be found. Please check the file path.")
    try:
        video = mp.VideoFileClip(video_path)
        audio_path = "extracted_audio.wav"
        if video.audio is not None:
            video.audio.write_audiofile(audio_path)
            video.reader.close()
            video.audio.reader.close_proc()
            with wave.open(audio_path, 'r') as wf:
                duration = wf.getnframes() / wf.getframerate()
                logging.info(f"Extracted audio duration: {duration} seconds")
            return audio_path
        else:
            logging.warning("No audio stream found in video.")
            return None
    except Exception as e:
        logging.error(f"Error extracting audio: {e}")
        return None



In [8]:
def audio_to_spectrograms(audio_path, sample_rate=16000, num_spectrograms=10):
    if audio_path is None:
        return []
    audio_tensor = load_wav_16k_mono(audio_path)
    spectrograms = []
    interval = 10 
    num_samples = sample_rate * interval
    total_segments = max(len(audio_tensor) // num_samples, 1)  

    if num_spectrograms > total_segments:
        num_spectrograms = total_segments
    
    selected_segments = sorted(random.sample(range(total_segments), num_spectrograms))

    for i in selected_segments:
        start = i * num_samples
        end = min(len(audio_tensor), start + num_samples)
        y_segment = audio_tensor[start:end]
        
        if len(y_segment) < num_samples:
            y_segment = tf.pad(y_segment, [[0, num_samples - len(y_segment)]])
        
        stft = tf.signal.stft(y_segment, frame_length=256, frame_step=128)
        spectrogram = tf.abs(stft)
        spectrogram = tf.expand_dims(spectrogram, axis=-1)
        spectrograms.append(spectrogram)
    return spectrograms

# Actual Prediction

In [13]:
def predict_video(video_path, threshold=0.5):
    AUDIO_MODEL_PATH = r'E:\FYP\DeepFake-Detection\DeepFake-Audio-Detection\Models\DFDM-M3.keras'
    IMAGE_MODEL_PATH = r'E:\FYP\DeepFake-Detection\DeepFake-Image-Detection\models\DFDM-FM.keras'

    try:
        audio_model = tf.keras.models.load_model(AUDIO_MODEL_PATH, custom_objects=custom_object)
        image_model = tf.keras.models.load_model(IMAGE_MODEL_PATH, custom_objects=custom_object)
        logging.info("Models loaded successfully.")
    except Exception as e:
        logging.error(f"Error loading the models: {e}")
        raise

    frames = extract_frames(video_path, num_frames=10)
    audio_path = extract_audio(video_path)
    spectrograms = audio_to_spectrograms(audio_path, sample_rate=16000, num_spectrograms=10)

    frame_predictions = []
    frame_images = []
    for i, frame in enumerate(frames):
        preprocessed_frame = preprocess_image(frame)
        prediction = image_model(preprocessed_frame)
        frame_predictions.append(prediction[0][0].numpy())  # Keep as float

        # Convert frame to base64 string for transmission
        _, buffer = cv2.imencode('.jpg', frame)
        frame_str = base64.b64encode(buffer).decode('utf-8')
        frame_images.append(frame_str)

    audio_predictions = []
    spectrogram_images = []
    for i, spectrogram in enumerate(spectrograms):
        preprocessed_spectrogram = preprocess_spectrogram(spectrogram)
        prediction = audio_model(preprocessed_spectrogram)
        audio_predictions.append(prediction[0][0].numpy())  # Keep as float

        # Convert spectrogram to base64 string for transmission
        plt.imshow(np.squeeze(spectrogram), cmap='inferno', aspect='auto')
        plt.axis('off')
        buf = BytesIO()
        plt.savefig(buf, format='png')
        plt.close()
        buf.seek(0)
        spectrogram_str = base64.b64encode(buf.getvalue()).decode('utf-8')
        spectrogram_images.append(spectrogram_str)

    avg_frame_prediction = np.mean(frame_predictions)
    avg_audio_prediction = np.mean(audio_predictions)
    #final_prediction = (avg_frame_prediction + avg_audio_prediction) / 2

    frame_predictions_formatted = [f"Frame Prediction: {pred:.2f} - {'Real' if pred >= threshold else 'Fake'}" for pred in frame_predictions]
    audio_predictions_formatted = [f"Audio Prediction: {pred:.2f} - {'Real' if pred >= threshold else 'Fake'}" for pred in audio_predictions]
    #final_prediction_formatted = f"Final Prediction: {final_prediction:.2f} - {'Real' if final_prediction >= threshold else 'Fake'}"
    avg_frame_prediction_formatted = f" {avg_frame_prediction:.2f} - {'Real' if avg_frame_prediction >= threshold else 'Fake'}"
    avg_audio_prediction_formatted = f" {avg_audio_prediction:.2f} - {'Real' if avg_audio_prediction >= threshold else 'Fake'}"

    try:
        os.remove(audio_path)
    except (PermissionError, TypeError) as e:
        logging.error(f"Error removing audio file: {e}")

    return frame_images, frame_predictions_formatted, spectrogram_images, audio_predictions_formatted, avg_frame_prediction_formatted, avg_audio_prediction_formatted

In [10]:
#video_path = r'C:\Users\user\Desktop\DeepFake-Detection\WP_Video.mp4'

#final_prediction = predict_video(video_path)

#print(f"Final Prediction: {final_prediction}")

In [14]:
app = Flask(__name__)
CORS(app)

@app.route('/predict', methods=['POST'])
def predict():
    try:
        if 'video' not in request.files:
            return jsonify({"error": "No video file provided"}), 400

        video_file = request.files['video']
        video_path = "temp_video.mp4"
        video_file.save(video_path)

        frame_images, frame_predictions_formatted, spectrogram_images, audio_predictions_formatted, avg_frame_prediction_formatted, avg_audio_prediction_formatted = predict_video(video_path)

        result = {
            "frame_images": frame_images,
            "frame_predictions": frame_predictions_formatted,
            "spectrogram_images": spectrogram_images,
            "audio_predictions": audio_predictions_formatted,
            "Video": avg_frame_prediction_formatted,
            "Audio":avg_audio_prediction_formatted
        }
    except Exception as e:
        logging.error(f"Error during prediction: {e}")
        return jsonify({"error": str(e)}), 500
    finally:
        if os.path.exists(video_path):
            os.remove(video_path)

    return jsonify(result)

In [15]:
if __name__ == '__main__':
    try:
        logging.info("Starting Flask server...")
        app.run(debug=True, host='0.0.0.0', port=5000, use_reloader=False)
    except Exception as e:
        logging.error(f"Error starting Flask server: {e}")

INFO:root:Starting Flask server...


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.13:5000
INFO:werkzeug:Press CTRL+C to quit
  warnings.warn(

  saveable.load_own_variables(weights_store.get(inner_path))

INFO:root:Models loaded successfully.


MoviePy - Writing audio in extracted_audio.wav


INFO:root:Extracted audio duration: 7.52 seconds                   


MoviePy - Done.


DEBUG:matplotlib.pyplot:Loaded backend module://matplotlib_inline.backend_inline version unknown.
DEBUG:matplotlib.pyplot:Loaded backend module://matplotlib_inline.backend_inline version unknown.
DEBUG:matplotlib.font_manager:findfont: Matching sans\-serif:style=normal:variant=normal:weight=normal:stretch=normal:size=10.0.
DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='c:\\Users\\user\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\matplotlib\\mpl-data\\fonts\\ttf\\STIXSizTwoSymBol.ttf', name='STIXSizeTwoSym', style='normal', variant='normal', weight=700, stretch='normal', size='scalable')) = 10.335
DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='c:\\Users\\user\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\matplotlib\\mpl-data\\fonts\\ttf\\DejaVuSerifDisplay.ttf', name='DejaVu Serif Display', style='normal', variant='normal', weight=400, stretch='normal', size='scalable')) = 10.05
DEBUG:matplotlib.font_manager:findfo